# Chapter 3 - Effective Functions

## 3.1 Python's Functions are First-Class

* **Everything in Python is an object**, including functions. You can
assign them to variables, store them in data structures, and pass
or return them to and from other functions (first-class functions.)
* First-class functions allow you to abstract away and pass
around behavior in your programs.
* Functions can be nested and they can capture and carry some
of the parent function’s state with them. Functions that do this
are called closures.
* **Objects can be made callable**. In many cases this allows you to
treat them like functions.


In [75]:
# function for use in this section
def yell(text):
    return text.upper()+ '!'

### functions and references

In [76]:
# We can make a reference that copies the function object
bark = yell

In [77]:
bark('woof')

'WOOF!'

In [78]:
del yell

In [79]:
yell('hello')

NameError: name 'yell' is not defined

In [80]:
bark('hey')

'HEY!'

In [81]:
# string identifier at creation time
bark.__name__

'yell'

### functions can be stored in data structures

In [82]:
funcs = [bark, str.lower, str.capitalize]

funcs

[<function __main__.yell(text)>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

In [83]:
# accessing functions inside data structures
for f in funcs:
    print(f, f('hey there'))

<function yell at 0x00000174EEA5BA60> HEY THERE!
<method 'lower' of 'str' objects> hey there
<method 'capitalize' of 'str' objects> Hey there


In [84]:
funcs[0]('yo')

'YO!'

### functions can be passed to other functions

* Functions that accept other functions as arguments are also called higher-order functions
* the ```map``` function is an example of a higher order function

In [85]:
list(map(bark,['hello', 'hey','hi']))

['HELLO!', 'HEY!', 'HI!']

In [86]:
def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)

In [87]:
# pass bark to greet function
greet(bark)

HI, I AM A PYTHON PROGRAM!


In [88]:
def whisper(text):
    return text.lower() + '...'

In [89]:
greet(whisper)

hi, i am a python program...


### functions can be nested

* Notice that ```whisper``` does not exist outside ```speak```

In [90]:
def speak(text):
    def whisper(t):
        return t.lower()+'...'
    return whisper(text)

In [91]:
speak('Hello, World')

'hello, world...'

### functions can return behaviours!

In [92]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper

In [93]:
speak_func = get_speak_func(0.7)

In [94]:
speak_func('Hello')

'HELLO!'

In [95]:
get_speak_func(0.2)('Hello')

'hello...'

### functions can capture local state

*  Inner functions can capture and carry some of the parent function's state
* This mean functions can pre-configure returned behaviours

### lexical closures

In [96]:
# notice the inner functions don't have any inputs
# they look in the parents scope for the variables
def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper

In [97]:
get_speak_func('Hello world', 0.7)()

'HELLO WORLD!'

#### preconfigured behaviour 

In [98]:
def make_adder(n):
    def add(x):
        # nested function captures the
        # state of the parent variable n         
        return x + n
    return add

In [99]:
plus_3 = make_adder(3)

In [100]:
plus_3(4)

7

In [101]:
plus_5 = make_adder(5)
plus_5(4)

9

### Objects can behave like functions

* All functions are objects in Python
* The reverse is not true!
* However, __objects can be made callable__ which allows you to treat them like functions in many cases

We do this through the use of the ```__call__``` dunder method

In [103]:
# define a class Adder and use the __call__ dunder to make it callable
# this is a good illustration where lexical closure is more efficient

class Adder:
    def __init__(self,n):
        self.n = n
    
    # the dunder method allows us to call this object
    def __call__(self,x):
        return self.n + x
    
# instantiate the class Adder
plus_3 = Adder(3)
# call the object like a function
plus_3(4)

7

* Note, not all objects will be callable, so we can check with the ```callable``` check

In [104]:
callable(plus_3)

True

In [105]:
callable('string')

False

In [106]:
callable(bark)

True

___
## 3.2 Lambdas are Single-Expression Functions

* Lambda functions are **single-expression functions** that are not
necessarily bound to a name (anonymous).
* Lambda functions can’t use regular Python statements and **always include an implicit return statement**.
* Always ask yourself: Would using a regular (named) function
or a list comprehension offer more clarity?
* **Saving a few keystrokes won’t matter in the long run, but your colleagues (and your future self) will appreciate clean and readable code
more than terse wizardry.**


In [112]:
add = lambda x, y: x + y
add(5,3)

8

* this is equivalent to the more verbose...

In [113]:
def add(x,y):
    return x + y

In [114]:
add(5,3)

8

### the power of lambda functions - function expressions

In [115]:
# function expressions - inline function definitions

(lambda x, y: x + y)(5,3)

8

### lambdas you can use

#### sorting keys

In [116]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
sorted(tuples, key=lambda x: x[1])

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

In [117]:
sorted(range(-5, 6), key=lambda x: x * x)

[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]

#### they work as lexical closures

In [118]:
def make_adder(n):
    # the state of the parent function (n) is captured by the lambda function     
    return lambda x: x + n 

plus_3 = make_adder(3)
plus_5 = make_adder(5)

In [119]:
plus_3(4)

7

In [120]:
plus_5(4)

9

### Take care writing lambdas functions because there are some pitfalls

* Ask yourself does this lambda function make your code maintainable and reusable?

In [121]:
# harmful
# this will be confusing as a way to write class methods
class Car:
    rev = lambda self: print('Wroom!')
    crash = lambda self: print('Boom!')
my_car = Car()

my_car.rev()
my_car.crash()

Wroom!
Boom!


In [122]:
# better
class Car:
    def rev(self):
        print('Wroom!')
    def crash(self):
        print('Boom!')
        
my_car = Car()
my_car.crash()

Boom!


In [123]:
# harmful
list(filter(lambda x: x % 2 == 0, range(16)))

[0, 2, 4, 6, 8, 10, 12, 14]

In [124]:
# better
[x for x in range(16) if x % 2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

___
## 3.3 The Power of Decorators

At their core, Python’s **decorators allow you to extend and modify the
behavior of a callable** (functions, methods, and classes) **without permanently modifying the callable itself**. 

**A decorator is a callable that takes a callable as input and
returns another callable**


### Key summary

* Decorators define reusable building blocks you can apply to a
callable to modify its behavior without permanently modifying
the callable itself. It also let's you call the functions by their original name. 
* The ```@``` syntax is just a shorthand for calling the decorator on
an input function. Multiple decorators on a single function are
applied **bottom to top (decorator stacking)**.
* As a debugging best practice, use the ```functools.wraps``` helper
in your own decorators to **carry over metadata from the undecorated callable to the decorated one**.
* Just like any other tool in the software development toolbox,
decorators are not a cure-all and they should not be overused.
It’s important to balance the need to “get stuff done” with the
goal of “not getting tangled up in a horrible, unmaintainable
mess of a code base.”
They "decorate" another function and let you execute code before and after the wrapped function runs. 

### Uses of decorators

* logging
* enforcing access control and authentication
* instrumentation and timing functions
* rate-limiting
* caching, and more

### Why use decorators? 

They allow you to define reusable building blocks that can change or extend the behavior of other functions. 

In [186]:
# very simple decorator ie. it does nothing

# we take a callable as input
def null_decorator(func):
    # we need to return a callable 
    return func

def greet():
    return 'hello'

In [187]:
greet = null_decorator(greet)
greet()

'hello'

Instead of explicitly calling ```null_decorator``` on ```greet``` you can make use of Python's @ syntax for decorating a function more conveniently. 

In [188]:
# the @null_decorator is just syntactic sugar
@null_decorator
def greet():
    return 'HELLO'

greet()

'HELLO'

### Decorators can modify behavior 

* The original callable isn't permanently modified - it's behavior changes only when decorated
* This let's you **add reusable building blocks** to existing functions and classes

In [194]:
# A general decorator

# a callable that accepts and returns a callable
def uppercase(func):     
    def wrapper(word):
        original_result = func(word)
        modified_result = original_result.upper()
        return modified_result 
    return wrapper

In [195]:
# remember this is syntactic sugar for uppercase(greet())
@uppercase 
def greet(word):
    return word + '!'

In [196]:
greet('hello')

'HELLO!'

### Applying Multiple Decorators to a Function 

* You can apply more then one decorator
* This is what makes decorators so powerful

In [199]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

In [200]:
# conventional function wrapping
def greet():
    return 'hello'

decorated_greet = strong(emphasis(greet))
decorated_greet()

'<strong><em>hello</em></strong>'

In [201]:
# decorator stacking is more readible
@strong
@emphasis
def greet():
    return 'Hello!'

In [202]:
# calling this decorated function clearly shows the order of application
greet()

'<strong><em>Hello!</em></strong>'

### Decorating functions that accept a variable number of arguments

* Python's ```*args``` and ```**kwargs``` are great for use with decorators

In [203]:
# It is better to use the *args, **kwargs instead of being explicit
def proxy(func):
    def wrap(*args, **kwargs):
        return func(*args, **kwargs)
    return wrap

Below is a decorator that is very versatile because of the use of ```*args``` and ```**kwargs```.

In [210]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}()  '
              f'with {args}, {kwargs}')
        
        function_result = func(*args,**kwargs)
        
        print(f'TRACE: {func.__name__}()  '
              f'returned {function_result!r}')
        
        return function_result
    return wrapper

In [211]:
@trace 
def say(name, line):
    return f'{name}: {line}'

In [212]:
say('Jane', 'says hello world')

TRACE: calling say()  with ('Jane', 'says hello world'), {}
TRACE: say()  returned 'Jane: says hello world'


'Jane: says hello world'

In [213]:
@trace 
def add(a,b):
    return f'The result of {a} + {b} is {a+b}'

In [214]:
add(3,4)

TRACE: calling add()  with (3, 4), {}
TRACE: add()  returned 'The result of 3 + 4 is 7'


'The result of 3 + 4 is 7'

### Writing Debuggable Decorators

When you use a decorator, really what you’re doing is replacing one
function with another. One downside of this process is that it “hides”
some of the metadata attached to the original (undecorated) function.

In [215]:
# A general decorator
def uppercase(func):     
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result 
    return wrapper

def greet():
    """Return a friendly greeting."""
    return 'Hello!'

decorated_greet = uppercase(greet)

In [216]:
greet.__name__

'greet'

In [217]:
greet.__doc__

'Return a friendly greeting.'

In [218]:
decorated_greet.__name__

'wrapper'

In [219]:
# this doesn't return anything
decorated_greet.__doc__

The work around is the ```functools.wraps``` decorator included in Python's standard library. This will **copy over the lost metadata from the undecorated function to the decorator closure**

In [220]:
import functools 

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

In [221]:
@uppercase
def greet():
    """Introduction message"""
    return 'Hello!'

In [222]:
# we retain the original name of the function
greet.__name__

'greet'

In [223]:
# now we get the wrapped functions doc string
greet.__doc__

'Introduction message'

___
## 3.4 Fun with ```*args``` and ```**kwargs```

### Summary 
* ```*args``` and ```**kwargs``` let you write functions with a **variable
number of arguments** in Python.
* ```*args``` collects extra positional arguments as a **tuple**. ```**kwargs```
collects the extra keyword arguments as a **dictionary**.
* The actual syntax is * and **. Calling them args and kwargs is
just a convention (and one you should stick to).


In [225]:
# required variable is a compulsary input
# the following positional arguments and keyword arguments are optional
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)

In [226]:
foo('hello')

hello


In [227]:
foo('hello', 1, 2, 3)

hello
(1, 2, 3)


In [228]:
foo('hello', 1, 2, 3, k1 = 'car', k2 = 10)

hello
(1, 2, 3)
{'k1': 'car', 'k2': 10}


### Forwarding Optional or Keyword Arguments

It's possible to pass optional or keyword parameters from one function to another. You can do this by unpacking ```*args``` and ```**kwargs```

In [233]:
def bar(x, *args, **kwargs):
    print(x)
    if args: 
        print(args)
    if kwargs:
        print(kwargs)

# kwargs and args are forwarded to bar
def foo(x, *args, **kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + ('extra', )
    bar(x, *new_args, **kwargs)

In [234]:
foo('x', 1, 2, 3, key1='car', key2='bus')

x
(1, 2, 3, 'extra')
{'key1': 'car', 'key2': 'bus', 'name': 'Alice'}


### Subclassing

__NOTE__ : this is a common design pattern in tkinter

You can use it to **extend the behavior of a parent class** without having to replicate the full signature of its constructor in the child class. We pass on the args and kwargs to the parent constructor

In [235]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

class AlwaysBlueCar(Car):
    def __init__(self, *args, **kwargs):
        # we pass on all arguments to the parent class
        super().__init__(*args, **kwargs)
        self.color = 'blue'

In [236]:
car = AlwaysBlueCar('red', 28392)
car.color

'blue'

**TAKE CARE**


However, you should be careful as the child class has an *unhelpful signature* since we don't know what arguments it expects without looking up the parent class!

### Wrapper functions

Ideal use case as you typically **want to pass an arbitrary number of arguments** to the wrapped function. 

You need to **walk the line** between: 

**EXPLICIT VS DRY (DON'T REPEAT YOURSELF)**

In [237]:
import functools
from datetime import datetime

In [265]:
# decorators
def trace(f):
    @functools.wraps(f)
    def decorated_function(*args,**kwargs):
        print('[trace]', f, args, kwargs)
        return f(*args, **kwargs)
        
    return decorated_function

def timer(f):
    @functools.wraps(f)
    def decorated_function(*args,**kwargs):
        print('[timer]', datetime.now())
        return f(*args, **kwargs)
    
    return decorated_function  

In [284]:
@trace
@timer
def greet(greeting,name, *args, **kwargs):
    """Prints a greeting"""
    return '{}, {}!'.format(greeting, name)

@trace
@timer
def logger(name, *args, **kwargs):
    """Logs information"""
    return f'Please wait {name} logging information'

In [285]:
greet('Hello', 'Bob', age = 21)

[trace] <function greet at 0x00000174EFCBCF70> ('Hello', 'Bob') {'age': 21}
[timer] 2021-05-04 16:26:44.616408


'Hello, Bob!'

In [286]:
logger('Thomas', 20,'6292-2907', postcode = '2904', city = 'New York')

[trace] <function logger at 0x00000174EFCBCA60> ('Thomas', 20, '6292-2907') {'postcode': '2904', 'city': 'New York'}
[timer] 2021-05-04 16:26:44.669229


'Please wait Thomas logging information'

In [287]:
logger.__doc__

'Logs information'

___
## 3.5 Function Argument Unpacking

### Summary
* We can use the ```*``` and ```**``` operators to unpack function arguments from sequences and dictionaries.
*  Using argument unpacking effectively can help you write more
flexible interfaces for your modules and functions.
* They can help **reduce complexity** by removing the need to build custom data structures

In [288]:
# simple example to work with
def print_vector(x,y,z):
    print('<%s, %s, %s>' % (x,y,z))

In [289]:
print_vector(0,1,0)

<0, 1, 0>


In [290]:
tuple_vector = (0,1,0)

In [291]:
# making use of the splat operator
print_vector(*tuple_vector)

<0, 1, 0>


### Use with generators

In [292]:
g = (x*x for x in range(3))

# we can use this splat operator with generators
# fully unpacking a generator
print_vector(*g)

<0, 1, 4>


### Use with dictionaries

In [293]:
dict_vec = { 'x' : 1, 'y' : 5, 'z' : -10}

#### ```* operator```

In [294]:
# keys will get passed in a random order
print_vector(*dict_vec)

<x, y, z>


#### ```** operator```

In [295]:
# the kwargs are passed to the function print_vector
# within print_vector's scope x, y and z map to the keys of the dictionary 
print_vector(**dict_vec)

<1, 5, -10>


#### Subtle differences

In [296]:
def print_vals(*args, **kwargs):
    if args:
        for arg in args:
            print(arg)
            
    if kwargs:
        print(kwargs)
        for kwarg in kwargs:
            print(f'{kwarg} : {kwargs[kwarg]}')

In [297]:
# an object
print_vals(dict_vec)

{'x': 1, 'y': 5, 'z': -10}


In [298]:
# unpack keywords
print_vals(*dict_vec)

x
y
z


In [299]:
# unpack kwargs
print_vals(**dict_vec)

{'x': 1, 'y': 5, 'z': -10}
x : 1
y : 5
z : -10


___
## 3.6 Nothing to Return Here

### Summary 

* **Rule of thumb: leave out return statement if the encapsulated code doesn't actually return anything**. The other case is encapsulated code where we are only interested in the side-effects.
* Other languages name an encapsulated portion of code a ***procedure* if there is no return statement**
* If a function doesn't specify a return value, it returns ```None```
Whether to return ```None``` is a stylistic decision
* This is a core Python feature but your code might communicate its intent more clearly with an explicit ```return None``` statement

In [300]:
# all 3 imply the same behavior
def foo1(value):
    if value:
        return value
    else:
        return None

# I tend to gravitate towards this function definition
def foo2(value):
    """Bare return statement implies 'return None'"""
    if value:
        return value 
    else:
        return 
    
# can be considered a procedure
def foo3(value):
    """Missing return statement"""
    if value:
        return value

In [301]:
# passing falsy values
print(type(foo1(0)))
print(type(foo2(0)))
print(type(foo3(0)))

<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
